In [83]:
import requests
import base64
import json
import pandas as pd
import logging
import os
from datetime import datetime
import sys

base_url = 'https://jira.footlocker.com/rest/api/2/search'

In [84]:

def get_report(base_url):
    logger.info("Getting report results...")
    header_gs = {'Accept': 'application/json'}
    headers = { 'Authorization' : 'Bearer %s' %  'OTQ5MjQwODU4MTU4OtHnNkWQEBeFpg4UlD91PxlqSR+H'}
    # 'Content-Length':3000 }
    logger.info(base_url)
    
    for i in range(0,3):
        r = requests.get(base_url,headers=headers)
        logger.info(r)
        if r.ok:
            logger.info("Report results received...")
            logger.info("HTTP %i - %s" % (r.status_code, r.reason))
            return r.text
        else:
            logger.error("HTTP %i - %s" % (r.status_code, r.reason))
            continue
    return None

In [91]:
def getDataSet(query):
    pd_obj = None
    maxRslt = 2000
    totalRsltFetch = -1
    df= pd.DataFrame()
    while True:
        dataset= get_report(base_url+query+f"&maxResults={maxRslt}&startAt={totalRsltFetch + 1 }")
        if dataset != None:
            obj = json.loads(dataset)
            logger.info(f"total: {obj['total']}")
            totalRsltFetch = totalRsltFetch + maxRslt

            if obj['issues']:
                df = pd.concat([df, pd.json_normalize(obj['issues']) ], ignore_index=True)
            else:
               logger.error(f'No records found....')
            if (totalRsltFetch >= obj['total'] ):
                break
        # pd_obj = pd.json_normalize(obj['issues'])
            

    return df
    

In [86]:
def getUrlDataSet(url):
    pd_obj = None

    dataset= get_report(url)
    if dataset != None:
        pd_obj = json.loads(dataset)
        # pd_obj = pd.json_normalize(obj['issues'])
    return pd_obj

In [87]:
def setup_logger( level=logging.INFO):
    """Configures a logger and returns it."""

    # if not os.path.exists('./log'):
    #     os.makedirs('./log')

    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

    # handler = logging.FileHandler('./log/soleProj.log')
    handler = logging.StreamHandler()
    handler.setFormatter(formatter)
   
    logger = logging.getLogger()
    logger.setLevel(level)
    
    if (logger.hasHandlers()):
        logger.handlers.clear()

    logger.addHandler(handler)
   

    return logger

In [88]:

def sprintvalue(x,field):
    if(len(x)!=0):
      return x[field]
    else:
     return 'NA'

In [89]:
def keyvalue(x,field):
    if(len(x)!=0):
      return x[0][field]
    else:
     return 'NA'


In [90]:

def convert_to_key_value(string):
    result = {}
    # print(string)
    if string :
        pairs = string[0].split(',')  # Split string by comma
        for pair in pairs:
            key, value = pair.split('=')  # Split each pair by '='
            result[key.strip()] = value.strip()  # Add to dictionary, stripping whitespace
    return result